In [53]:
from pedalboard import Pedalboard, Reverb, load_plugin
from pedalboard.io import AudioFile
from mido import Message

In [ ]:
# Open Vital plugin
vital = load_plugin("/home/ben/Synthetizers-parameters-estimation/plugins/Vital.vst3")
print(vital.parameters.keys())

In [ ]:
# Play a note: C4 (MIDI note 60) with velocity 0.8
sample_rate = 44100
audio = vital(
  [Message("note_on", note=60), Message("note_off", note=60, time=5)],
  duration=5, # seconds
  sample_rate=sample_rate,
)

from IPython.display import Audio
Audio(audio, rate=sample_rate)

In [ ]:
import json

file = 'preset.vital'
with open(file, '+r') as f:
    preset_data = json.load(f)

In [ ]:
print(preset_data.keys())

dict_keys(['author', 'comments', 'macro1', 'macro2', 'macro3', 'macro4', 'preset_name', 'preset_style', 'settings', 'synth_version'])


In [ ]:
print(preset_data['settings'].keys())

dict_keys(['beats_per_minute', 'bypass', 'chorus_cutoff', 'chorus_delay_1', 'chorus_delay_2', 'chorus_dry_wet', 'chorus_feedback', 'chorus_frequency', 'chorus_mod_depth', 'chorus_on', 'chorus_spread', 'chorus_sync', 'chorus_tempo', 'chorus_voices', 'compressor_attack', 'compressor_band_gain', 'compressor_band_lower_ratio', 'compressor_band_lower_threshold', 'compressor_band_upper_ratio', 'compressor_band_upper_threshold', 'compressor_enabled_bands', 'compressor_high_gain', 'compressor_high_lower_ratio', 'compressor_high_lower_threshold', 'compressor_high_upper_ratio', 'compressor_high_upper_threshold', 'compressor_low_gain', 'compressor_low_lower_ratio', 'compressor_low_lower_threshold', 'compressor_low_upper_ratio', 'compressor_low_upper_threshold', 'compressor_mix', 'compressor_on', 'compressor_release', 'delay_aux_frequency', 'delay_aux_sync', 'delay_aux_tempo', 'delay_dry_wet', 'delay_feedback', 'delay_filter_cutoff', 'delay_filter_spread', 'delay_frequency', 'delay_on', 'delay_sty

In [ ]:
print(len(set(preset_data["settings"].keys())))
print(len(vital.parameters.keys()))

776
772


In [ ]:
# Number of keys in intersection
common_keys = set(preset_data["settings"].keys()).intersection(set(vital.parameters.keys()))
print(len(common_keys))

519


In [ ]:
# Show keys in preset but not in vital parameters
preset_only_keys = set(preset_data["settings"].keys()) - set(vital.parameters.keys())
print(preset_only_keys)

# Show keys in vital parameters but not in preset
vital_only_keys = set(vital.parameters.keys()) - set(preset_data["settings"].keys())
print(vital_only_keys)

{'osc_2_destination', 'osc_1_spectral_morph_amount', 'env_2_attack', 'chorus_spread', 'lfo_8_keytrack_tune', 'osc_3_pan', 'osc_3_spectral_unison', 'env_5_decay_power', 'filter_1_keytrack', 'osc_3_unison_blend', 'modulations', 'random_2_sync', 'reverb_high_shelf_cutoff', 'lfo_5_delay_time', 'env_2_delay', 'env_6_sustain', 'compressor_band_upper_threshold', 'env_1_release', 'env_2_attack_power', 'random_3_sync', 'random_1_tempo', 'lfo_4_keytrack_transpose', 'osc_1_level', 'eq_on', 'reverb_low_shelf_cutoff', 'osc_2_stereo_spread', 'env_2_decay_power', 'lfo_3_keytrack_tune', 'osc_3_distortion_type', 'osc_1_random_phase', 'random_1_stereo', 'macro_control_1', 'env_6_attack_power', 'env_3_decay', 'osc_3_unison_voices', 'osc_2_level', 'filter_1_on', 'random_1_sync', 'osc_3_level', 'env_1_delay', 'lfo_7_fade_time', 'lfo_4_keytrack_tune', 'compressor_low_lower_ratio', 'flanger_on', 'env_3_release_power', 'osc_2_distortion_phase', 'osc_1_spectral_morph_type', 'env_5_release', 'osc_2_spectral_mor

In [ ]:
CORE_COMPONENT_MAP = {
    # Component Mapping (Prefixes)
    "osc": "oscillator",
    "env": "envelope",
    "random": "random_lfo",
    
    # Macro Control Mapping
    "macro_control": "macro",
    
    # Effect Mapping
    "reverb": "reverb",
    "chorus": "chorus",
    "phaser": "phaser",
    "flanger": "flanger",
    "eq": "eq",
    "delay": "delay",
    "compressor": "compressor", # Stays 'compressor' in both, though Set 2 uses specific band terms
    
    # Other Core Modules
    "sample": "sample",
    "wavetables": "wavetables",
    "lfos": "lfos",
    "modulations": "modulations"
}

COMMON_SUFFIX_MAP = {
    # On/Off
    "_on": "_switch",
    
    # Envelope Parameter Mapping
    "_attack": "_attack",
    "_decay": "_decay",
    "_sustain": "_sustain",
    "_release": "_release",
    "_hold": "_hold",
    
    # LFO Keytracking
    "_keytrack_tune": "_tune",
    "_keytrack_transpose": "_transpose",
    "_fade_time": "_fade_in",
    
    # Oscillator Specific
    "_spectral_morph": "_frequency_morph",
    "_random_phase": "_phase_randomization",
    "_frame_spread": "_frame_spread", # Note: Set 2 often uses "unison_frame_spread" for this context
    
    # Effect Mix
    "_dry_wet": "_mix",
    
    # Filter/Effect Keytracking
    "_keytrack": "_key_track",
}

preset_new_settings = {}
for key, value in preset_data["settings"].items():
    new_key = key
    for prefix, mapped_prefix in CORE_COMPONENT_MAP.items():
        if key.startswith(prefix):
            new_key = key.replace(prefix, mapped_prefix, 1)
            break
    for suffix, mapped_suffix in COMMON_SUFFIX_MAP.items():
        if new_key.endswith(suffix):
            new_key = new_key[:-len(suffix)] + mapped_suffix
            break
    preset_new_settings[new_key] = value

# Print common length after remapping
common_keys_remapped = set(preset_new_settings.keys()).intersection(set(vital.parameters.keys()))
print(len(common_keys_remapped))

# Show keys in preset but not in vital parameters after remapping
preset_only_keys_remapped = set(preset_new_settings.keys()) - set(vital.parameters.keys())
print(preset_only_keys_remapped)

# Show keys in vital parameters but not in preset after remapping
vital_only_keys_remapped = set(vital.parameters.keys()) - set(preset_new_settings.keys())
print(vital_only_keys_remapped)

723
{'delay_aux_sync', 'compressor_high_upper_ratio', 'lfo_7_delay_time', 'oscillator_3_spectral_morph_spread', 'lfos', 'oscillator_1_spectral_morph_type', 'chorus_spread', 'oscillator_3_frame_spread', 'oscillator_1_spectral_morph_spread', 'sample_phase_randomization', 'oscillator_3_spectral_morph_amount', 'modulations', 'oscillator_1_spectral_morph_amount', 'reverb_high_shelf_cutoff', 'delay_aux_tempo', 'lfo_6_delay_time', 'lfo_4_delay_time', 'lfo_5_delay_time', 'compressor_band_upper_threshold', 'sample', 'compressor_low_upper_threshold', 'compressor_low_upper_ratio', 'sample_key_track', 'reverb_low_shelf_cutoff', 'wavetables', 'compressor_high_lower_threshold', 'delay_aux_frequency', 'filter_fx_blend_transpose', 'lfo_8_delay_time', 'reverb_high_shelf_gain', 'oscillator_1_unison_blend', 'compressor_band_lower_threshold', 'oscillator_2_unison_blend', 'filter_2_blend_transpose', 'lfo_2_delay_time', 'oscillator_3_spectral_morph_type', 'chorus_cutoff', 'oscillator_3_unison_blend', 'compr

In [ ]:
# All types using getattr(vital,key).type
from collections import Counter
compteur = Counter()
for key in common_keys_remapped:
    compteur[getattr(vital,key).type] += 1
print(compteur)

Counter({<class 'float'>: 326, <class 'bool'>: 240, <class 'str'>: 157})


In [ ]:
# All types using getattr(vital,key).type
from collections import Counter
compteur = Counter()
for key in common_keys_remapped:
    if getattr(vital,key).type == str:
        print(f"{key}, {getattr(vital,key)}, {preset_new_settings[key]}")
    if getattr(vital,key).type == bool:
        print(f"{key}, {getattr(vital,key)}, {preset_new_settings[key]}")
print(compteur)

modulation_28_bipolar, False, 0.0
eq_high_cutoff, 100 semitones, 123.26066589355469
modulation_52_bipolar, False, 0.0
modulation_41_bypass, False, 0.0
envelope_3_decay, 1 secs, 1.0
lfo_4_frequency, 0.5 secs, 1.0
envelope_1_release, 0.0899194 secs, 0.7910539507865906
envelope_4_delay, 0 secs, 0.0
modulation_51_stereo, False, 0.0
random_lfo_4_sync_type, False, 0.0
lfo_3_frequency, 0.5 secs, 1.0
modulation_62_bipolar, False, 0.0
modulation_11_bipolar, False, 0.0
envelope_2_delay, 0 secs, 0.0
distortion_filter_cutoff, 80 semitones, 101.12000274658203
modulation_31_bipolar, False, 0.0
random_lfo_4_sync, Tempo, 1.0
modulation_9_stereo, False, 0.0
modulation_16_bypass, False, 0.0
modulation_42_bipolar, False, 0.0
modulation_36_bipolar, False, 0.0
random_lfo_1_sync_type, False, 0.0
modulation_48_stereo, False, 0.0
lfo_3_tempo, 1/2, 7.0
modulation_48_bypass, False, 0.0
modulation_27_stereo, False, 0.0
modulation_49_bypass, False, 0.0
modulation_21_stereo, False, 0.0
modulation_32_bypass, False,

In [ ]:
# Play with the plugin parameters
sound = vital(
  [Message("note_on", note=10), Message("note_off", note=10, time=5)],
  duration=5,
  sample_rate=sample_rate,
)
Audio(sound, rate=sample_rate)

In [ ]:
# For all of these keys in common_keys_remapped, set the vital parameter to the preset value
failed = 0
success = 0
skipped = 0

for key in common_keys_remapped:
    try:
        param_type = getattr(vital, key).type
        
        # Skip string parameters
        if param_type == str:
            skipped += 1
            continue
        
        # Convert value based on parameter type
        value = preset_new_settings[key]
        
        if param_type == bool:
            if isinstance(value, str):
                value = value.lower() in ['true', '1', '1.0', 'yes']
            else:
                value = bool(value)
        else:
            # Handle numeric types (float/int)
            value = float(value) if param_type == float else value
        
        # Set the parameter
        setattr(vital, key, value)
        success += 1
        
    except Exception as e:
        print(f"Failed to set {key}: {e}")
        failed += 1

print(f"Skipped: {skipped}, Failed: {failed}, Success: {success}")

Failed to set filter_fx_formant_resonance: Value received for parameter 'filter_fx_formant_resonance' (0.8500000238418579) is out of range [30.0%, 100.0%]
Failed to set chorus_delay_2: Value received for parameter 'chorus_delay_2' (-7.0) is out of range [0.976562ms, 19.9999ms]
Failed to set filter_1_formant_resonance: Value received for parameter 'filter_1_formant_resonance' (0.8500000238418579) is out of range [30.0%, 100.0%]
Failed to set chorus_delay_1: Value received for parameter 'chorus_delay_1' (-9.0) is out of range [0.976562ms, 19.9999ms]
Failed to set filter_2_formant_resonance: Value received for parameter 'filter_2_formant_resonance' (0.8500000238418579) is out of range [30.0%, 100.0%]
Failed to set volume: Value received for parameter 'volume' (5473.04052734375) is out of range [-80.0dB, 6.02dB]
Failed to set chorus_voices: Value received for parameter 'chorus_voices' (1.0) is out of range [4.0None, 16.0None]
Failed to set beats_per_minute: Value received for parameter 'be

In [ ]:
# Play with the plugin parameters
sound = vital(
  [Message("note_on", note=10), Message("note_off", note=10, time=5)],
  duration=5,
  sample_rate=sample_rate,
)
Audio(sound, rate=sample_rate)